# Import Modules

In [1]:
import pandas as pd

# Import Data

In [2]:
precovid = pd.read_csv('raw_data/escooters-precovid.csv', index_col = 'TripID')
postcovid = pd.read_csv('raw_data/escooters-postcovid.csv', index_col = 'TripID')

In [3]:
df = precovid.append(postcovid)

# Remove NA

In [4]:
df.isnull().sum()

StartDate         0
StartTime         0
EndDate           4
EndTime           4
TripDuration      0
TripDistance      0
StartLatitude     0
StartLongitude    0
EndLatitude       0
EndLongitude      0
DayOfWeek         0
HourNum           0
dtype: int64

In [5]:
df.dropna(inplace = True)

# Invalid Values

Let's get the indexes of the invalid values:

In [6]:
invalid = df.loc[(df.StartTime.str.contains('24:')) | (df.EndTime.str.contains('24:'))]

Replace the invalid value '24' with '00':

In [7]:
for col in ['StartTime', 'EndTime']:
    df.loc[invalid.index, col] = df.loc[invalid.index, col].str.replace('24', '00')

Merge the `StartDate` with `StartTime` and `EndDate` with `EndTime`:

In [8]:
df['Start'] = df['StartDate'].str.cat(df['StartTime'], sep = ' ')

In [9]:
df['End'] = df['StartDate'].str.cat(df['EndTime'], sep = ' ')

Convert them to datetime:

In [10]:
for col in ['Start', 'End']:
    df[col] = pd.to_datetime(df[col])

Then we have to add a day to the `Start`, as travels started at `24:00` on one day are actually starting at `00:00` of the following one!

In [11]:
invalid_start = df.loc[ df.StartTime.str.contains('24:') ].index

In [12]:
df.loc[invalid_start, 'StartTime'] = df.loc[invalid_start, 'StartTime'] + pd.Timedelta(days = 1)

# Reorder columns

In [13]:
df.drop(axis = 1,
        columns = ['DayOfWeek', 'HourNum', 'StartTime', 'StartDate', 'EndTime', 'EndDate'],
        inplace = True)

In [14]:
df.rename(columns = { 'TripDuration' : 'Duration',
                     'TripDistance' : 'Distance'},
          inplace = True)

# Export the data

In [15]:
df.to_csv('data/escooters-merged.csv')